In [6]:
!wget https://github.com/AlertaDengue/PySUS/raw/master/pysus/Notebooks/Getting_CNES_Data.ipynb

--2022-03-17 09:51:38--  https://github.com/AlertaDengue/PySUS/raw/master/pysus/Notebooks/Getting_CNES_Data.ipynb
Resolvendo github.com (github.com)... 20.201.28.151
Conectando-se a github.com (github.com)|20.201.28.151|:443... conectado.
A requisição HTTP foi enviada, aguardando resposta... 302 Found
Localização: https://raw.githubusercontent.com/AlertaDengue/PySUS/master/pysus/Notebooks/Getting_CNES_Data.ipynb [redirecionando]
--2022-03-17 09:51:39--  https://raw.githubusercontent.com/AlertaDengue/PySUS/master/pysus/Notebooks/Getting_CNES_Data.ipynb
Resolvendo raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Conectando-se a raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... conectado.
A requisição HTTP foi enviada, aguardando resposta... 200 OK
Tamanho: 47374 (46K) [text/plain]
Salvando em: “Getting_CNES_Data.ipynb”

Getting_CNES_Data.i 100%[===================>]  46,26K  --.-KB/s    em 0,008s  

In [1]:
from datetime import datetime
from tempfile import NamedTemporaryFile
import geopandas as gpd
import os
from ftplib import FTP
from pysus.utilities.readdbc import read_dbc, dbc2dbf
from dbfread import DBF
import pandas as pd
from pysus.online_data import CACHEPATH


group_dict = {
        "LT" :  ["Leitos - A partir de Out/2005", 10, 2005],
        "ST" :  ["Estabelecimentos - A partir de Ago/2005", 8, 2005],
        "DC" :  ["Dados Complementares - A partir de Ago/2005", 8, 2005],
        "EQ" :  ["Equipamentos - A partir de Ago/2005", 8],
        "SR" :  ["Serviço Especializado - A partir de Ago/2005", 8, 2005],
        "HB" :  ["Habilitação - A partir de Mar/2007", 3, 2007],
        "PF" :  ["Profissional - A partir de Ago/2005", 8, 2005],
        "EP" :  ["Equipes - A partir de Abr/2007", 5, 2007],
        "IN" :  ["Incentivos - A partir de Nov/2007", 11, 2007],
        "RC" :  ["Regra Contratual - A partir de Mar/2007", 3, 2007], 
        "EE" :  ["Estabelecimento de Ensino - A partir de Mar/2007", 3, 2007],
        "EF" :  ["Estabelecimento Filantrópico - A partir de Mar/2007", 3, 2007],
        "GM" :  ["Gestão e Metas - A partir de Jun/2007",  6, 2007]

}

def download(group: str, state: str, year: int, month: int, cache: bool=True) -> object:
    """
    Download CNES records for group, state, year and month and returns dataframe
    :param group: 
        LT – Leitos - A partir de Out/2005
        ST – Estabelecimentos - A partir de Ago/2005
        DC - Dados Complementares - A partir de Ago/2005
        EQ – Equipamentos - A partir de Ago/2005
        SR - Serviço Especializado - A partir de Ago/2005
        HB – Habilitação - A partir de Mar/2007
        PF – Profissional - A partir de Ago/2005
        EP – Equipes - A partir de Abr/2007
        IN – Incentivos - A partir de Nov/2007
        RC - Regra Contratual - A partir de Mar/2007
        EE - Estabelecimento de Ensino - A partir de Mar/2007
        EF - Estabelecimento Filantrópico - A partir de Mar/2007
        GM - Gestão e Metas - A partir de Jun/2007
    :param month: 1 to 12
    :param state: 2 letter state code
    :param year: 4 digit integer
    """
    state = state.upper()
    year2 = str(year)[-2:]
    month = str(month).zfill(2)
    input_date = datetime(int(year), int(month), 1)
    avaiable_date = datetime(group_dict[group][2], group_dict[group][1], 1)
    if input_date < avaiable_date:
        raise ValueError(f"CNES does not contain data for {group_dict[group][1]}")
    ftp = FTP('ftp.datasus.gov.br')
    ftp.login()
    if input_date >= avaiable_date:
        ftype = 'DBC'
        ftp.cwd('dissemin/publicos/CNES/200508_/Dados/{}/'.format(group))
        fname = '{}{}{}{}.dbc'.format(group, state, str(year2).zfill(2), month)
    cachefile = os.path.join(CACHEPATH, 'CNES_' + fname.split('.')[0] + '_.parquet')
    if os.path.exists(cachefile):
        df = pd.read_parquet(cachefile)
        return df
    df = _fetch_file(fname, ftp, ftype)
    if cache:
        df.to_parquet(cachefile)
    return df


def _fetch_file(fname, ftp, ftype):
    try:
        ftp.retrbinary('RETR {}'.format(fname), open(fname, 'wb').write)
    except:
        raise Exception("File {} not available".format(fname))
    if ftype == 'DBC':
        df = read_dbc_geopandas(fname, encoding='iso-8859-1')
    elif ftype == 'DBF':
        dbf = DBF(fname, encoding='iso-8859-1')
        df = pd.DataFrame(list(dbf))
    os.unlink(fname)
    return df


def read_dbc_geopandas(filename,encoding='utf-8'):
    """
    Opens a DATASUS .dbc file and return its contents as a pandas
    Dataframe, using geopandas
    :param filename: .dbc filename
    :param encoding: encoding of the data
    :return: Pandas Dataframe.
    """
    if isinstance(filename, str):
        filename = filename
    with NamedTemporaryFile(delete=False) as tf:
        out = tf.name + '.dbf'
        dbc2dbf(filename, out)
        dbf = gpd.read_file(out, encoding=encoding).drop("geometry",axis=1)
        df = pd.DataFrame(dbf)
    os.unlink(tf.name)

    return df 

In [2]:
df = download('SR','SC', 2022, 2)

In [3]:
df

,CNES,CODUFMUN,SERV_ESP,CLASS_SR,SRVUNICO,REGSAUDE,MICR_REG,DISTRSAN,DISTRADM,TPGESTAO,...,CNPJ_MAN,CARACTER,AMB_NSUS,AMB_SUS,HOSP_NSUS,HOSP_SUS,COMPETEN,CONTSRVU,CNESTERC,NAT_JUR
0,0275166,420005,173,001,173,008,8ª,None,7ª,M,...,78511052000110,1,0,1,0,0,202202,1,None,1244
1,0275166,420005,173,002,None,008,8ª,None,7ª,M,...,78511052000110,1,0,1,0,0,202202,None,None,1244
2,0275166,420005,174,001,174,008,8ª,None,7ª,M,...,78511052000110,1,0,1,0,0,202202,1,None,1244
3,0275166,420005,174,002,None,008,8ª,None,7ª,M,...,78511052000110,1,0,1,0,0,202202,None,None,1244
4,0875791,420005,115,002,115,010,None,None,None,M,...,00000000000000,1,0,1,0,0,202202,1,None,3999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41669,9821597,422000,145,009,None,None,None,None,None,M,...,00000000000000,1,0,1,0,0,202202,None,None,2062
41670,9821597,422000,145,010,None,None,None,None,None,M,...,00000000000000,1,0,1,0,0,202202,None,None,2062
41671,9821597,422000,145,011,None,None,None,None,None,M,...,00000000000000,1,0,1,0,0,202202,None,None,2062
41672,9821597,422000,145,012,None,None,None,None,None,M,...,00000000000000,1,0,1,0,0,202202,None,None,2062


In [16]:
df = download('ST','SC', 2022, 2)

In [35]:
len(df.CNES.unique())

21450

In [18]:
INTERESSE = ["CNES", "CODUFMUN", "COD_CEP", "CPF_CNPJ", "PF_PJ", "REGSAUDE", "MICR_REG", "VINC_SUS", "TPGESTAO", "ESFERA_A", "ATIVIDAD", "NATUREZA", "COMPETEN" ]

In [19]:
df[INTERESSE]

,CNES,CODUFMUN,COD_CEP,CPF_CNPJ,PF_PJ,REGSAUDE,MICR_REG,VINC_SUS,TPGESTAO,ESFERA_A,ATIVIDAD,NATUREZA,COMPETEN
0,0275166,420005,89636000,00000000000000,3,008,8ª,1,M,None,04,None,202202
1,0875791,420005,89636000,02757271000111,3,010,None,1,M,None,04,None,202202
2,2380218,420005,89636000,00000000000000,3,008,8ª,1,M,None,04,None,202202
3,6425208,420005,89636000,00000000000000,3,7,None,1,M,None,04,None,202202
4,7250355,420005,89636000,02478749000255,3,8,None,1,M,None,04,None,202202
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21445,9197133,422000,88828000,14456105000164,3,None,None,0,M,None,04,None,202202
21446,9197141,422000,88828000,03222124000100,3,None,None,0,M,None,04,None,202202
21447,9775390,422000,88828000,00000000000000,3,None,None,1,M,None,04,None,202202
21448,9821597,422000,88828000,31393852000110,3,None,None,1,M,None,04,None,202202


In [20]:
df.REGSAUDE.unique()

array(['008', '010', '7', '8', None, '01', '001', '5.º', '5.ª', '5',
       '005', '12', '4ª', '004', '12º', '4', '7ª', '8ª', 'JBA', '10',
       '18', '18ª', '13', '1ª', '1º', '1', '15', '27ª', '018', '9¬', '14',
       '3', '003', '22', '23', '6', '006', '0006', '06', '22ª', '20',
       '20ª', '02', '09', '15ª', '007', '17', '07', '017', '11', '1SDR',
       '17º', ',', '16', '26', '9', '35', '0', '8901', '27', '28', '5º',
       '36ª', '36º', '36', '46', '9ª', '002', '0002', '2', '4210', '4202',
       '29', '25', '0008', '08', '16º', '16ª', '26ª', '7º', '8º', '12ª',
       '21', '012', '24', '2º', '05', '5ª', '34', '0009', 'XXE', '21ª',
       '9ª G', '9G', '9 G', '9º G', '3ª', '18 R', '20º', '009', '0017',
       '024', '04', '14ª', 'R14', '035', '31º', '0001', '03', 'VI', '33',
       '0011', '011', '25ª', '11º', '7°', '17ª', '31', '13ª', '13A.',
       '14§', '19', '8 º', '0003', '04ª', '33ª', '4207', '17ª.', '016',
       '29º', '29ª', '28º', '21 ª', '4º', '3º', '21º', '18º', 

In [21]:
df.MICR_REG.unique()

array(['8ª', None, '8', '12', '7ª', 'CHAPEC', 'N', 'AMOSC', 'MACRO',
       'LAGES', '13', '000000', '0', '02', '7', '22', '006', '06', '1',
       '03', '15', '27', '29', 'AMARP', '000', '00000', '14', '10',
       'AMAI', '01', '0000', '17', 'SC', 'SMO', 'EXT', 'EXTR', 'MICRO',
       '00', '18', '0014', '24', '000008', 'JOACAB', '31', '1º', '1ª',
       'AMAUC', '25º', '11', '04', '05', '420910', '42012', '421000',
       'AMVALI', 'AMVALE', '25', '421105', 'AMREC', '008', 'EXTREM',
       'OESTE2', '003', '001', '004', '002', '005', '009', '192', '011',
       '012', '42013', '14ª', '12ª', '16', 'TIMBO', '6', '421570',
       '42.009', '23', 'NORTE', '5', '33', '21', 'CARBON', '20', '13ª',
       '4', '0001'], dtype=object)

In [22]:
with open("listaCEPS.txt", "w") as fp:
    for x in df.COD_CEP.unique():
        fp.write(x + "\n")

In [24]:
df[INTERESSE].sample(n=100).to_csv("sample.csv")

In [32]:
df[INTERESSE].groupby("ESFERA_A").count()

,CNES,CODUFMUN,COD_CEP,CPF_CNPJ,PF_PJ,REGSAUDE,MICR_REG,VINC_SUS,TPGESTAO,ATIVIDAD,NATUREZA,COMPETEN
ESFERA_A,,,,,,,,,,,,


In [33]:
df.ESFERA_A

0        None
1        None
2        None
3        None
4        None
         ... 
21445    None
21446    None
21447    None
21448    None
21449    None
Name: ESFERA_A, Length: 21450, dtype: object

In [40]:
df.to_csv("STSC202202.csv",index=False)

In [39]:
df.TP_UNID.unique()

array(['84', '36', '02', '68', '39', '81', '74', '22', '43', '05', '70',
       '04', '40', '01', '78', '42', '73', '80', '07', '77', '71', '50',
       '72', '85', '15', '60', '62', '76', '75', '83', '20', '21', '82',
       '69', '32', '79'], dtype=object)